# Quick Start: Machine Learning

Content for my website: <a href="https://sdiehl28.netlify.com/" target="_blank">Software Nirvana</a>

### Create a Skeleton
Start with a [skeleton][1] of your project and then fill in the details.  The idea is to quickly get something that works, end-to-end and then [Kiazen](https://en.wikipedia.org/wiki/Kaizen) the model to improve its accuracy.

Having something that works end-to-end will alert you to what types of problems are inherent in your particular dataset and will provide a rough sense of the accuracy that can be expected.
[1]: https://en.wikipedia.org/wiki/Skeleton_(computer_programming)

For tutorial purposes, each iteration will do the bare minimum.  Someone comfortable with the machine learning workflow would likely do more in each iteration, however they would still start with a simple model whose accuracy is used as a baseline.

### Machine Learning Example
Make a prediction for survived / not-survived using the titanic dataset from Kaggle.  This is a supervised learning problem.

Notebooks that follow will iteratively Kiazen this one.

### Outline
1. [Acquire and Read Data](#readdata)
2. [Identify Target Variable](#target)
3. [Train / Test Split](#traintest)
4. [Exploratory Data Analysis](#eda)
5. [Preprocessing](#preprocess)
6. [Model Creation](#model)
7. [Model Evaluation](#eval)

### Common Imports and Notebook Setup

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
%matplotlib inline
sns.set() # enable seaborn style

### Check Software Versions

In [15]:
import sys
print('python:     ', sys.version)
print('numpy:      ', np.__version__)
print('pandas:     ', pd.__version__)
import matplotlib
print('matplotlib: ', matplotlib.__version__)
print('seaborn:    ', sns.__version__)
print('sklearn:    ', sk.__version__)

python:      3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
numpy:       1.13.3
pandas:      0.22.0
matplotlib:  2.1.2
seaborn:     0.8.1
sklearn:     0.19.1


<a name="readdata"></a>
### Acquire the Data

Download "train.csv" from: https://www.kaggle.com/c/titanic/data and place it in a data subdirectory.

### Read in Data

In [16]:
# read in all the labeled data
all_data = pd.read_csv('../data/train.csv')
all_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<a name="target"></a>
### Target Variable: Survived

In [17]:
# break up the dataframe into X and y
# X is a 2 dimensional "spreadsheet" of values used for prediction
# y is a 1 dimensional vector of target (aka response) values
X = all_data.drop('Survived', axis=1)
y = all_data['Survived']
print('X Shape: ', X.shape)
print('y Shape: ', y.shape)

X Shape:  (891, 11)
y Shape:  (891,)


<a name="traintest"></a>
### Train and Test Split
Do this prior to Exploratory Data Analysis and other Model Building Steps to avoid "leakage" of test data.

Train/Test split will later be refined to use cross validation.

Performing the same operation on the train and test datasets will later be refined to use pipeline.

SD TODO: provide link

In [18]:
# create the train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=111)

<a name="eda"></a>
### Exploratory Data Analysis
One of the first things to check for is **null values**.  In this first iteration of creating a machine learning model, this will be the only EDA performed.

In [19]:
# Find the percentage of missing values per column
nrows, ncols = X_train.shape
X_train.isnull().sum() / nrows

PassengerId    0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.187801
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.776886
Embarked       0.003210
dtype: float64

### Null Value Analysis
The following is a reasonable judgement call as to how to proceed based on the observed percentages of null values.
1. The Age attribute has some missing values => impute missing values
2. Most of the Cabin attribute is missing => remove it
3. Very few Emarked records are missing => remove records with missing Emarked value

Age imputation is likely to be helpful, however in this first iteration the goal is to quickly create a model and measure its accuracy for use as a baseline for Kaizening.

Write a note to remind ourself and others of this.
**TODO: Try Age Imputation**

In [20]:
# Discard Age column (for now)
X_train = X_train.drop('Age', axis=1)
X_test = X_test.drop('Age', axis=1)

In [21]:
# Discard Cabin column
X_train = X_train.drop('Cabin', axis=1)
X_test = X_test.drop('Cabin', axis=1)

### Examine Datatypes
Examine the datatypes.  Convert the datatype to categorical as necessary.

** TODO: convert the following to categorical **
* Pclass
* Sex
* Embarked


In [22]:
# For 1st Pass, ignore text and categorical variables
X_train = X_train.drop('Pclass', axis=1)
X_test = X_test.drop('Pclass', axis=1)
X_train = X_train.drop('Name', axis=1)
X_test = X_test.drop('Name', axis=1)
X_train = X_train.drop('Sex', axis=1)
X_test = X_test.drop('Sex', axis=1)
X_train = X_train.drop('Ticket', axis=1)
X_test = X_test.drop('Ticket', axis=1)
X_train = X_train.drop('Embarked', axis=1)
X_test = X_test.drop('Embarked', axis=1)

In [23]:
# Examine the datatypes of each remaining column
X_train.dtypes

PassengerId      int64
SibSp            int64
Parch            int64
Fare           float64
dtype: object

<a name="Preprocessing"></a>
### Preprocessing
Preprocessing was done "inline" with Exploratory Data Analysis above.

<a name="model"></a>
### Model Building
Perhaps the simplest model to try for classification is Logistic Regression.

Special techniques are required if one class is much more rare than another.  Let's check for that first.

In [24]:
y.value_counts()

0    549
1    342
Name: Survived, dtype: int64

That's close enough to equally distributed that Logistic Regression should produce a good result.

In [25]:
# Build Model
from sklearn.linear_model import LogisticRegression
base_model = LogisticRegression()
base_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

<a name="eval"></a>
### Model Evaluation
The simplest measure of accuracy is to look at the percent of correct predictions.

In [27]:
predictions = base_model.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predictions, y_test)

array([[154,  69],
       [ 17,  28]])

In [31]:
# Compute Accuracy
base_accuracy = (154 + 28) / (154+69+17+28)
print(base_accuracy)

0.6791044776119403


In [36]:
# Compare with Simplest Possible Model
# Prediction predominant class every time
y_test.value_counts()

0    171
1     97
Name: Survived, dtype: int64

In [38]:
# "Null" Model Accuracy
null_accuracy = 171 / (171 + 97)
print(null_accuracy)

0.6380597014925373


### Conclusion
The simplest model had a prediction accuracy of about 68%.  The null model which just predicts the most common class in all cases was accurate about 64% of the time.

In this first iteration:
* we quickly created a model
* noted a few things to try next
* established a baseline accuracy of 68%
* showed that this accuracy is better than the null model accuracy of 64%